In [1]:
!wget https://github.com/omseervi098/sam/raw/main/Backend1/models/sam_vit_b_encoder.onnx

--2024-01-19 08:42:46--  https://github.com/omseervi098/sam/raw/main/Backend1/models/sam_vit_b_encoder.onnx
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/omseervi098/sam/main/Backend1/models/sam_vit_b_encoder.onnx [following]
--2024-01-19 08:42:46--  https://raw.githubusercontent.com/omseervi098/sam/main/Backend1/models/sam_vit_b_encoder.onnx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100270670 (96M) [application/octet-stream]
Saving to: 'sam_vit_b_encoder.onnx'

sam_vit_b_encoder.o 100%[===================>]  95.62M   237MB/s    in 0.4s    

2024-01-19 08:42:56 (237 MB/s) - 'sam_vit_b_en

In [2]:
!pip install flask flask-cors pillow numpy opencv-python-headless onnx onnxruntime flask-ngrok pyngrok==4.1.1

  Obtaining dependency information for flask-cors from https://files.pythonhosted.org/packages/10/69/1e6cfb87117568a9de088c32d6258219e9d1ff7c131abf74249ef2031279/Flask_Cors-4.0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/7a/cf/6aa8c56fd63f53c2c485921e411269c7b501a2b4e634bd02f226ab2d5d8e/onnxruntime-1.16.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 52.1 MB/s eta 0:00:00:00:010:01
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4340dfc4ee1155ceb9433f32b0561c1abf9ea1bec42a2bdceefe3966b0b0718b
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902

In [3]:
!ngrok authtoken "Add your ngrok token here"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml                                      


In [18]:
from flask_cors import CORS, cross_origin
from flask import Flask, send_file, request
from PIL import Image
import flask
import numpy as np
import cv2
import os
import onnxruntime
from flask_ngrok import run_with_ngrok

In [5]:
encoder_path = "/kaggle/input/onnxmodel/sam_vit_b_encoder.onnx"
encoder_session = onnxruntime.InferenceSession(encoder_path, providers=['CPUExecutionProvider'])

In [ ]:
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)
@app.route('/', methods=['GET'])
def home():
    return "Hello, World !!"

@app.route('/test',methods=['POST'])
def testpost():
    imagfile=flask.request.files['image']
    print(imagfile)
    return "Image Recieved"

@app.route('/getembedding', methods=['POST'])
def getembedding1():
    imagefile = flask.request.files['image']
    img = Image.open(imagefile)
    cv_image = np.array(img)
    input_size = (684, 1024)
    scale_x = input_size[1] / cv_image.shape[1]
    scale_y = input_size[0] / cv_image.shape[0]
    scale = min(scale_x, scale_y)
    transform_matrix = np.array(
        [
            [scale, 0, 0],
            [0, scale, 0],
            [0, 0, 1],
        ]
    )
    cv_image = cv2.warpAffine(
        cv_image,
        transform_matrix[:2],
        (input_size[1], input_size[0]),
        flags=cv2.INTER_LINEAR,
    )
    encoder_inputs = {
        "input_image": cv_image.astype(np.float32),
    }
    output = encoder_session.run(None, encoder_inputs)
    if output is not None:
        output = output[0]
    image_embedding = output
    response=flask.jsonify(image_embedding.tolist())
    return response


if (__name__ == '__main__'):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on http://491a-34-79-140-164.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
